In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import Request, urlopen
import json
import pandas as pd

In [ ]:
# Function to parse to web
def web_parser(link, headers = {'User-Agent': 'Mozilla/5.0'}):
    req = Request(
    url= link, 
    headers={'User-Agent': 'Mozilla/5.0'}
    )
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    return soup

In [ ]:
def get_link(soup):
    links = soup.find('article', class_='comp comp_reviews-pagination querylist-pagination position-').find('ul').find_all('a')
    new_url = 'https://www.airlinequality.com' + links[-1].get('href')
    return new_url

In [ ]:
def craw_data_review(soup):
    dataset = []
    count = 1
    while True:
        review_date = soup.find_all('meta', itemprop = 'datePublished')
        review_title = soup.find_all('h2', class_='text_header')
        review_text = soup.find_all('div', class_= 'text_content', itemprop='reviewBody')
        for i in range(len(review_text)):
            temp_rv_title = review_title[i].get_text(strip=True)
            temp_rv_date = review_date[i].get('content')
            if review_text[i].get_text(strip = True).find('| ') == -1:
                temp_review = review_text[i].get_text(strip = True)
                dataset.append({'Review date': temp_rv_date, 'Review title': temp_rv_title, 'Review': temp_review})
            else:
                temp_review = review_text[i].get_text(strip=True).split('| ')[1:]
                dataset.append({'Review date': temp_rv_date, 'Review title': temp_rv_title, 'Review': ' '.join(temp_review).strip()})
            
        link = get_link(soup)
        new_soup = web_parser(link)
        if new_soup == soup:
            print("Last page")
            break
        else:
            soup = new_soup
            print(f"Finish page {count}")
        count += 1
    return dataset





In [ ]:
req = Request(
    url= 'https://www.airlinequality.com/airline-reviews/british-airways', 
    headers={'User-Agent': 'Mozilla/5.0'}
)
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, 'html.parser')

In [ ]:
mydata = craw_data_review(soup)

In [ ]:
def convert(lst):
   res_dict = {}
   for i in range(0, len(lst), 2):
       res_dict[lst[i]] = lst[i + 1]
   return res_dict

In [ ]:
def crawl_data_stat(soup):
    review_stats = []
    count = 1
    while True:
        stats = soup.find_all('div', class_='review-stats')
        rv_date = soup.find_all('meta', itemprop = 'datePublished')
        for i in range(len(stats)):
            table = stats[i].find_all('td')
            temp_rv_date = rv_date[i].get('content')
            temp = []
            for j in range(0, len(table)):
                if table[j].get('class') == ['review-rating-stars', 'stars']:
                    if len(table[j].find_all('span', class_ ="star fill")) != 0:
                        value = table[j].find_all('span', class_ ="star fill")[-1].get_text()
                else:
                    value = table[j].get_text()
                temp.append(value)
            temp.append('Review date')
            temp.append(temp_rv_date)
            review_stats.append(convert(temp))
        link = get_link(soup)
        new_soup = web_parser(link)
        if new_soup == soup:
            print("Last page")
            break
        else:
            soup = new_soup
            print(f'Finish page {count}')
        count += 1
    return review_stats

        

In [ ]:
review_stats = crawl_data_stat(soup)

In [ ]:
dataframe = pd.DataFrame(review_stats)

In [ ]:
dataframe

In [ ]:
dataframe.to_csv('C:/Users/HP/OneDrive - National Economics University/Data Prep/Class code/review_stats_data.csv', encoding='utf-8')

In [ ]:
dataframe.to_stata('C:/Users/HP/OneDrive - National Economics University/Data Prep/Class code/review_stats_data.dta')

In [ ]:
pd.set_option('display.max_colwidth', None)
dataframe1 = pd.DataFrame(mydata)
dataframe1


In [ ]:
dataframe1.to_csv('C:/Users/HP/OneDrive - National Economics University/Data Prep/Class code/review_text_data.csv', encoding='utf-8')